inspiriert durch https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/examples/get_started_pytorch.py

In [6]:
from art.attacks.evasion import GRAPHITEWhiteboxPyTorch
from art.estimators.classification import PyTorchClassifier

from torchvision.transforms import ToTensor, Resize, Compose
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pytorch_ood.utils import ToRGB
from gtsrb import GTSRB
from pathlib import Path
from PIL import Image
import numpy as np

In [18]:
trans = Compose([
            ToRGB(),
            ToTensor(),
            Resize((64, 64), antialias=True)
        ])


batch_size = 3
test_data = GTSRB(root=".", train=False, transforms=trans)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

resnet18_model = torch.load("64x64/label-net-resnet18-64.pt", map_location="cpu", weights_only=False)
wideresnet40_model = torch.load("64x64/label-net-wrn40-64.pt", map_location="cpu", weights_only=False)

resnet18_model.eval()
wideresnet40_model.eval()


digital_imgs = Path("digital_imgs")
images = []
to_tensor = ToTensor()
for img in sorted(digital_imgs.glob("*.png")):
    img = Image.open(img)
    img = img.resize((64, 64))
    img_t = to_tensor(img)
    images.append(img_t)

images = torch.stack(images)
labels = [16, 1, 38, 11, 33, 18, 9, 13, 34, 2, 7, 13, 26, 15, 28]
labels = np.array(labels)

images.size()

torch.Size([15, 3, 64, 64])

testing on original images

In [19]:
with torch.no_grad():
    preds = wideresnet40_model(images).argmax(1)

accuracy = (preds == torch.tensor(labels)).float().mean().item()
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


splitten der Daten in train, val, test (5,5,5)

In [41]:
# vorher mischen, sonst bleibt die Reihenfolge
perm = torch.randperm(len(images))
images = images[perm]
labels = labels[perm.numpy()]

# Splits
train_imgs, train_labels = images[:5], labels[:5]
val_imgs,   val_labels   = images[5:10], labels[5:10]
test_imgs,  test_labels  = images[10:], labels[10:]

model = resnet18_model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(0.0, 1.0),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 64, 64),
    nb_classes=43,
)

#classifier.fit(train_imgs, train_labels, batch_size=batch_size, nb_epochs=3)

predictions = classifier.predict(val_imgs)
pred_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(pred_classes == val_labels)
print(f"Accuracy on benign val examples: {accuracy * 100:.2f}%")

Accuracy on benign val examples: 0.00%
